Generate imagery

In [1]:
# Convolutions!
# Order of images
# b4, b4_tex, b8, b8_tex, bs_asc, bs_desc, coh_asc, coh_desc, nl
# Order of truth
# id, fid, muni_code, volume, area, people

# Local path, change this.
yellow_follow = 'C:/Users/caspe/Desktop/yellow/lib/'

import sys; sys.path.append(yellow_follow) 
import sqlite3
import pandas as pd
import ml_utils
import numpy as np
import os

np.set_printoptions(suppress=True)
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

from sqlalchemy import create_engine

# Tensorflow
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Conv3D, MaxPooling3D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

In [2]:
folder = "C:/Users/caspe/Desktop/Paper_2_StructuralVolume/"

rotation = True
rotation_count = 4
epochs = 50
initial_learning_rate = 0.001
end_learning_rate = 0.00001

munis = {
    "Lemvig": 665,
    "Silkeborg": 740,
    "Aarhus": 751,
}

images = np.load(folder + "all_images.npy")
images_metadata = np.load(folder + "images_ground_truth.npy")
truth = np.load(folder + "structural_volume.npy")

In [13]:
test_muni_mask = (images_metadata[:, 2] == munis["Lemvig"])
train_muni_mask = (images_metadata[:, 2] != munis["Silkeborg"])

X_test = images[test_muni_mask]
y_test = truth[test_muni_mask]

zero_mask = (images_metadata[:, 3] > 0)[test_muni_mask]

X_train = images[train_muni_mask]
y_train = truth[train_muni_mask]

In [14]:
layers = [0, 2, 4, 5, 6, 7] # bsac_bsdc_s2
layer_name = "bsac_bsdc_s2"

# Selected layers
X_train = X_train[:, layers]
X_test = X_test[:, layers]

In [15]:
# Simple balance dataset (Equal amount 0 to rest)
frequency = ml_utils.count_freq(zero_mask)
minority = frequency.min(axis=0)[1]
balance_mask = ml_utils.minority_class_mask(zero_mask, minority)

X_train = X_train[balance_mask]
y_train = y_train[balance_mask]

In [17]:
if rotation is True:
    X_train = ml_utils.add_rotations(X_train, axes=(2, 3), k=rotation_count)
    y_train = ml_utils.add_rotations(y_train, axes=(1, 2), k=rotation_count)

In [18]:
# Shuffle the training dataset
shuffle_mask = np.random.permutation(len(y_train))
X_train = X_train[shuffle_mask]
y_train = y_train[shuffle_mask]

In [ ]:
def lr_time_based_decay(epoch):
    return initial_learning_rate - (epoch * ((initial_learning_rate - end_learning_rate) / epochs))

def define_optimizer():
    return tfa.optimizers.Lookahead(
        Adam(
            learning_rate=initial_learning_rate,
            name="Adam",
        )
    )

In [ ]:
def define_model(shape, name):
    drop = 0.2
    model_input = Input(shape=shape, name=name)
    model = Conv2D(64, kernel_size=3, padding='same', activation=tfa.activations.mish, kernel_initializer='he_uniform')(model_input)
    model = Conv2D(64, kernel_size=3, padding='same', activation=tfa.activations.mish, kernel_initializer='he_uniform')(model_input)
    model = BatchNormalization()(model)
    model = Dropout(drop)(model)

    model = MaxPooling2D(pool_size=(2, 2), padding='same', strides=(2, 2))(model)

    model = Conv2D(96, kernel_size=3, padding='same', activation=tfa.activations.mish, kernel_initializer='he_uniform')(model)
    model = BatchNormalization()(model)  
    model = Dropout(drop)(model)

    model = Conv2D(128, kernel_size=3, padding='same', activation=tfa.activations.mish, kernel_initializer='he_uniform')(model)
    model = BatchNormalization()(model)
    model = Dropout(drop)(model)

    model = conv2d_transpose(128, kernel_size=3, padding='same', strides=2, activation=tfa.activations.mish, kernel_initializer='he_uniform')(model)
    model = BatchNormalization()(model)
    model = Dropout(drop)(model)

    model = Conv2D(96, kernel_size=3, padding='same', activation=tfa.activations.mish, kernel_initializer='he_uniform')(model_input)
    model = BatchNormalization()(model)
    model = Dropout(drop)(model)

    model = Conv2D(64, kernel_size=3, padding='same', activation=tfa.activations.mish, kernel_initializer='he_uniform')(model_input)
    model = Conv2D(64, kernel_size=3, padding='same', activation=tfa.activations.mish, kernel_initializer='he_uniform')(model_input)

    return Model(inputs=[model_input], outputs=predictions)

In [ ]:
model.compile(
    optimizer=define_optimizer(),
    loss="mean_absolute_error",
)

model.fit(
    x=X_train,
    y=y_train,
    epochs=epochs,
    verbose=1,
    batch_size=512,
    validation_split=0.2,
    callbacks=[
        LearningRateScheduler(lr_time_based_decay, verbose=1),
        EarlyStopping(
            monitor="val_loss",
            patience=9,
            min_delta=1,
            restore_best_weights=True,
        ),
    ]
)

# Evaluate model
print("")
loss, mae = model.evaluate(X_test, y_test, verbose=2)
print("")
loss, z_mae = model.evaluate(X_test[zero_mask], y_test[zero_mask], verbose=2)
print("")